# Setup
## Parameters

In [1]:
label_classes = ['33+1', '8+1', '1+1']  # Classes: '33+1', '8+1', '1+1'

sampling_methods = ['None', 'RandomOverSampler', 'RandomUnderSampler', 'SMOTE', 'ClusterCentroids']   # Samplers: 'None', 'RandomOverSampler', 'RandomUnderSampler', 'SMOTE', 'ClusterCentroids'

evaluator_types = ['KNearestNeighbor']   # Evaluators: 'XGBoost', 'LogisticRegression', 'Perceptron', 'AdaBoost', 'RandomForest', 'DeepNeuralNetwork', 'KNearestNeighbor'

# Random Seeds
train_test_seed = 42
evaluator_seed = 42

# Import/Export Directories
metrics_directory = './metrics'
resampled_dataset_directory = './resampled_datasets'
evaluator_directory = './trained_evaluators'

# Notebook parameter validation

sampler_categories = ['None', 'RandomOverSampler', 'RandomUnderSampler', 'SMOTE', 'ClusterCentroids']
label_categories = ['33+1', '8+1', '1+1']
evaluator_categories = ['XGBoost', 'LogisticRegression', 'Perceptron', 'AdaBoost', 'RandomForest', 'DeepNeuralNetwork', 'KNearestNeighbor']

for label_class in label_classes:
    if label_class not in label_categories:
        assert False, f'{label_class} is an invalid class structure.'

for sampler in sampling_methods:
    if sampler not in sampler_categories:
        assert False, f'{sampler} is an invalid sampling method.'
    
for evaluator in evaluator_types:
    if evaluator not in evaluator_categories:
        assert False, f'{evaluator} is an invalid evaluator.'

## Common Packages

In [2]:
import os
import pandas as pd
import time
from IPython.display import display
from joblib import dump, load
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Importing the Dataset

In [3]:
all_filepaths = [filename for filename in os.listdir(resampled_dataset_directory) if filename.endswith('.parquet')]
print(f'CSVs in "{resampled_dataset_directory}": {len(all_filepaths)}\n')

datasets = {}

for sampler in sampling_methods:
    datasets[sampler] = {}
    for label_class in label_classes:
        print(f'Loading ./resampled_datasets/{sampler}_{label_class}_resampled_dataset.parquet')
        datasets[sampler][label_class] = pd.read_parquet(
            path=f'./resampled_datasets/{sampler}_{label_class}_resampled_dataset.parquet'
        )
        print(f'Dataset Shape: {datasets[sampler][label_class].shape}')
        display(datasets[sampler][label_class])

CSVs in "./resampled_datasets": 15

Loading ./resampled_datasets/None_33+1_resampled_dataset.parquet
Dataset Shape: (1389408, 47)


,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
0,4.625763,84.00,17.00,64.00,0.432361,0.432361,0.0,0.0,0.0,0.0,...,0.000000,42.00,8.300688e+07,9.5,9.165151,0.000000,0.000000e+00,0.00,141.55,21
1,76.135781,428611.80,8.20,151.90,42.352135,42.352135,0.0,0.0,0.0,0.0,...,1707.500922,655.00,1.665202e+08,13.5,41.470705,2419.498399,2.944407e+06,1.00,244.60,1
2,4.549627,108.00,6.00,64.00,0.762226,0.762226,0.0,0.0,1.0,0.0,...,0.000000,54.00,8.336142e+07,9.5,10.392305,0.000000,0.000000e+00,0.00,141.55,12
3,0.000000,54.00,6.00,64.00,0.000000,0.000000,0.0,0.0,1.0,0.0,...,0.000000,54.00,8.308992e+07,9.5,10.392305,0.000000,0.000000e+00,0.00,141.55,10
4,0.000000,0.00,1.00,64.00,3.178249,3.178249,0.0,0.0,0.0,0.0,...,0.000000,42.00,8.313213e+07,9.5,9.165151,0.000000,0.000000e+00,0.00,141.55,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233582,0.000000,54.00,6.00,64.00,5.966641,5.966641,0.0,0.0,0.0,0.0,...,0.000000,54.00,8.303339e+07,9.5,10.392305,0.000000,0.000000e+00,0.00,141.55,13
233583,8.547875,20177.97,15.59,68.64,7031.899016,7031.899016,0.0,0.0,0.0,0.0,...,68.440643,82.09,8.334834e+07,9.5,12.897158,96.738630,1.597739e+04,0.94,141.55,9
233584,0.000000,54.00,6.00,64.00,4.410794,4.410794,0.0,1.0,0.0,1.0,...,0.000000,54.00,8.334818e+07,9.5,10.392305,0.000000,0.000000e+00,0.00,141.55,9
233585,0.000000,0.00,1.00,64.00,22.793830,22.793830,0.0,0.0,0.0,0.0,...,0.000000,42.00,8.314974e+07,9.5,9.165151,0.000000,0.000000e+00,0.00,141.55,6


Loading ./resampled_datasets/None_8+1_resampled_dataset.parquet
Dataset Shape: (1389408, 47)


,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
0,4.625763,84.00,17.00,64.00,0.432361,0.432361,0.0,0.0,0.0,0.0,...,0.000000,42.00,8.300688e+07,9.5,9.165151,0.000000,0.000000e+00,0.00,141.55,3
1,76.135781,428611.80,8.20,151.90,42.352135,42.352135,0.0,0.0,0.0,0.0,...,1707.500922,655.00,1.665202e+08,13.5,41.470705,2419.498399,2.944407e+06,1.00,244.60,0
2,4.549627,108.00,6.00,64.00,0.762226,0.762226,0.0,0.0,1.0,0.0,...,0.000000,54.00,8.336142e+07,9.5,10.392305,0.000000,0.000000e+00,0.00,141.55,2
3,0.000000,54.00,6.00,64.00,0.000000,0.000000,0.0,0.0,1.0,0.0,...,0.000000,54.00,8.308992e+07,9.5,10.392305,0.000000,0.000000e+00,0.00,141.55,2
4,0.000000,0.00,1.00,64.00,3.178249,3.178249,0.0,0.0,0.0,0.0,...,0.000000,42.00,8.313213e+07,9.5,9.165151,0.000000,0.000000e+00,0.00,141.55,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233582,0.000000,54.00,6.00,64.00,5.966641,5.966641,0.0,0.0,0.0,0.0,...,0.000000,54.00,8.303339e+07,9.5,10.392305,0.000000,0.000000e+00,0.00,141.55,2
233583,8.547875,20177.97,15.59,68.64,7031.899016,7031.899016,0.0,0.0,0.0,0.0,...,68.440643,82.09,8.334834e+07,9.5,12.897158,96.738630,1.597739e+04,0.94,141.55,2
233584,0.000000,54.00,6.00,64.00,4.410794,4.410794,0.0,1.0,0.0,1.0,...,0.000000,54.00,8.334818e+07,9.5,10.392305,0.000000,0.000000e+00,0.00,141.55,2
233585,0.000000,0.00,1.00,64.00,22.793830,22.793830,0.0,0.0,0.0,0.0,...,0.000000,42.00,8.314974e+07,9.5,9.165151,0.000000,0.000000e+00,0.00,141.55,2


Loading ./resampled_datasets/None_1+1_resampled_dataset.parquet
Dataset Shape: (1389408, 47)


...

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
0,112.820531,2831.487500,11.118750,96.306250,23.680621,23.680621,0.0,0.0,0.0,0.0,...,45.542654,134.212500,2.385680e-02,5.5,15.369907,64.407038,3.311644e+03,0.856250,38.5,0
1,423.524072,7328.771429,8.357143,130.657143,76.308018,76.308018,0.0,0.0,0.0,0.0,...,119.951239,141.385714,1.676298e+08,13.5,16.618607,169.937605,1.560634e+04,1.000000,244.6,0
2,373.984638,83291.400000,8.200000,65.300000,65.517287,65.517287,0.0,0.0,0.0,0.0,...,3544.017932,3263.400000,4.300907e-03,5.5,100.956805,5011.998224,1.404716e+07,0.900000,38.5,0
3,4.749069,119489.200000,6.000000,64.000000,92.875204,92.875204,0.0,0.0,0.0,0.0,...,2087.442576,2350.800000,1.676297e+08,13.5,54.080281,2958.907948,4.402649e+06,1.000000,244.6,0
4,1.108356,95326.200000,6.000000,70.000000,62.912492,62.912492,0.0,0.0,0.0,0.0,...,2645.413247,351.600000,1.676298e+08,13.5,50.244107,3748.404394,7.065778e+06,1.000000,244.6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1083,0.471143,81146.200000,6.000000,226.000000,175.097529,175.097529,0.0,0.0,0.0,0.0,...,489.434923,1351.200000,1.676391e+08,13.5,51.043853,681.978236,2.339504e+05,1.000000,244.6,33
1084,0.964338,77485.000000,6.000000,52.800000,74.126744,74.126744,0.0,0.0,0.0,0.0,...,839.028208,637.200000,1.676390e+08,13.5,51.809400,1185.684037,7.063765e+05,1.000000,244.6,33
1085,1130.109987,69645.260000,7.540000,88.020000,17.557112,17.557112,0.0,0.0,0.0,0.0,...,72.770909,154.240000,1.676391e+08,13.5,15.379787,102.916058,6.505454e+03,0.960000,244.6,33
1086,835.037838,65703.500000,8.200000,123.300000,12.169314,12.169314,0.0,0.0,0.0,0.0,...,375.776075,118.700000,1.676391e+08,13.5,23.465460,532.431031,1.425352e+05,1.000000,244.6,33


Loading ./resampled_datasets/ClusterCentroids_8+1_resampled_dataset.parquet
Dataset Shape: (3224, 47)


,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
0,43.815188,2.718021e+06,6.547619,114.886905,142.163258,142.163258,0.0,0.0,0.0,1.355253e-20,...,658.614308,984.957143,1.665217e+08,13.5,38.301460,927.845058,4.631108e+05,0.997619,244.6,0
1,29.787825,8.510525e+05,7.081366,155.334161,573.360184,573.360184,0.0,0.0,0.0,-2.032879e-20,...,64.438030,274.290683,7.380798e-03,5.5,18.080104,91.129136,2.067982e+04,0.559938,38.5,0
2,22.785047,6.673545e+05,7.028378,113.526351,103.422655,103.422655,0.0,0.0,0.0,-6.776264e-21,...,481.688400,707.765541,1.665218e+08,13.5,29.922482,679.533278,2.757185e+05,0.972973,244.6,0
3,29.623289,5.007870e+05,6.958333,112.175000,159.085461,159.085461,0.0,0.0,0.0,0.000000e+00,...,2978.115790,1196.250000,1.665221e+08,13.5,60.897104,4218.946112,9.092597e+06,1.000000,244.6,0
4,8.284695,2.468985e+05,8.200000,202.400000,30.080409,30.080409,0.0,0.0,0.0,0.000000e+00,...,4155.947420,1312.600000,3.216478e-02,5.5,66.276102,5877.397206,1.974094e+07,0.900000,38.5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3219,674.457357,7.849950e+03,11.150000,72.200000,11.176992,11.176992,0.0,0.0,0.0,0.000000e+00,...,111.798171,218.000000,1.818210e-02,5.5,21.944259,158.106489,1.401305e+04,0.900000,38.5,7
3220,23.529895,1.009150e+03,6.800000,80.500000,120.401771,120.401771,0.0,0.0,0.0,0.000000e+00,...,61.602241,111.150000,1.676298e+08,13.5,15.036101,87.214399,3.806562e+03,1.000000,244.6,7
3221,872.579318,1.041210e+04,11.000000,95.400000,10.074352,10.074352,0.0,0.0,0.0,0.000000e+00,...,79.346773,196.600000,1.676298e+08,13.5,16.175083,112.542588,6.590925e+03,1.000000,244.6,7
3222,40.305533,9.581750e+02,9.725000,105.200000,37.461488,37.461488,0.0,0.0,0.0,0.000000e+00,...,55.779626,127.825000,1.295140e-02,5.5,15.342622,78.884304,3.951636e+03,0.800000,38.5,7


Loading ./resampled_datasets/ClusterCentroids_1+1_resampled_dataset.parquet
Dataset Shape: (65644, 47)


,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
0,9.248562e-02,29377.990170,16.940114,64.375852,8513.369067,8513.369067,-4.235165e-22,-1.387779e-17,5.551115e-17,-4.163336e-17,...,0.301614,50.203295,8.310671e+07,9.500000,10.012404,0.426270,8.672647e+00,0.028068,141.550000,0
1,1.183356e+01,973420.400000,17.000000,38.000000,81.536986,81.536986,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,255.675388,648.500000,7.186234e-04,5.500000,22.304492,361.579601,1.321211e+05,0.500000,38.500000,0
2,6.453561e+00,877661.200000,17.000000,57.700000,167.154275,167.154275,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,17.249820,1140.600000,1.668613e+08,13.500000,49.072889,5.634970,1.599328e+01,1.000000,244.600000,0
3,6.198883e-08,565.580000,6.770000,65.680000,19358.326154,19358.326154,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,488.856927,698.350000,1.000034e+08,9.500000,36.869890,691.428106,2.721642e+05,0.890000,141.550000,0
4,1.068042e-01,116406.603774,16.094340,64.000000,728.055153,728.055153,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,540.073609,900.358491,6.291813e+07,8.886792,43.460264,763.905060,3.093446e+05,0.943396,122.283019,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65639,2.220006e+01,34473.500000,13.700000,98.800000,46.190235,46.190235,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,163.796590,150.900000,3.051305e-03,5.500000,18.550145,231.643360,3.503273e+04,0.900000,38.500000,1
65640,3.811979e+01,32584.900000,12.600000,82.700000,7.859036,7.859036,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,48.033592,113.300000,1.665200e+08,13.500000,12.564913,68.144351,2.568164e+03,1.000000,244.600000,1
65641,3.419141e+01,67355.400000,6.000000,166.100000,26.202975,26.202975,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,357.930387,670.000000,1.665256e+08,13.500000,19.239078,508.534770,2.359663e+05,1.000000,244.600000,1
65642,2.665170e+01,27319.900000,6.000000,113.200000,49.856850,49.856850,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,523.426226,365.000000,6.194711e-03,5.500000,38.191663,740.236468,3.431003e+05,0.900000,38.500000,1


# Preprocessing
## Scaling Numerical Features

In [4]:
num_cols = [
    'flow_duration', 'Header_Length',  'Duration', 'Rate', 'Srate', 'ack_count', 'syn_count', 'fin_count',
    'urg_count', 'rst_count', 'Tot sum', 'Min', 'Max', 'AVG', 'Std', 'Tot size', 'IAT', 'Number', 'Magnitue',
    'Radius', 'Covariance', 'Variance', 'Weight'
]

scalers = {}
print('Scaled Datasets:')
for sampler in sampling_methods:
    scalers[sampler] = StandardScaler()

    for label_class in label_classes:
        datasets[sampler][label_class][num_cols] = scalers[sampler].fit_transform(
            datasets[sampler][label_class][num_cols]
        )
    
        print(f'Sampler: {sampler} / Label Class: {label_class}')
        display(datasets[sampler][label_classes[0]].head(3))

Scaled Datasets:
Sampler: None / Label Class: 33+1


,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
0,-0.003675,-0.166147,17.0,-0.167976,-0.090509,-0.090509,0.0,0.0,0.0,0.0,...,-0.208794,-0.343574,-0.011406,0.000835,-0.459184,-0.208627,-0.095484,-0.414344,0.000754,21
1,0.281530,0.760145,8.2,6.105171,-0.090092,-0.090092,0.0,0.0,0.0,0.0,...,10.488495,2.198032,4.882094,4.877978,3.281350,10.509058,9.134320,3.870082,4.885560,1
2,-0.003979,-0.166095,6.0,-0.167976,-0.090505,-0.090505,0.0,0.0,1.0,0.0,...,-0.208794,-0.293820,0.009369,0.000835,-0.317097,-0.208627,-0.095484,-0.414344,0.000754,12


Sampler: None / Label Class: 8+1


,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
0,-0.003675,-0.166147,17.0,-0.167976,-0.090509,-0.090509,0.0,0.0,0.0,0.0,...,-0.208794,-0.343574,-0.011406,0.000835,-0.459184,-0.208627,-0.095484,-0.414344,0.000754,21
1,0.281530,0.760145,8.2,6.105171,-0.090092,-0.090092,0.0,0.0,0.0,0.0,...,10.488495,2.198032,4.882094,4.877978,3.281350,10.509058,9.134320,3.870082,4.885560,1
2,-0.003979,-0.166095,6.0,-0.167976,-0.090505,-0.090505,0.0,0.0,1.0,0.0,...,-0.208794,-0.293820,0.009369,0.000835,-0.317097,-0.208627,-0.095484,-0.414344,0.000754,12


Sampler: None / Label Class: 1+1


...

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
0,0.158755,-0.460666,11.118750,0.380183,-0.231754,-0.231754,0.0,0.0,0.0,0.0,...,-0.588789,-0.797205,-1.461175,-1.502993,-0.863778,-0.588342,-0.231288,0.637615,-1.503092,0
1,1.524459,-0.457775,8.357143,1.501086,-0.231678,-0.231678,0.0,0.0,0.0,0.0,...,-0.490582,-0.785343,1.553021,1.532797,-0.790863,-0.489910,-0.228472,0.995545,1.540535,0
2,1.306708,-0.408941,8.200000,-0.631581,-0.231694,-0.231694,0.0,0.0,0.0,0.0,...,4.028637,4.377559,-1.461175,-1.502993,4.133899,4.026441,2.985967,0.746550,-1.503092,0


Sampler: ClusterCentroids / Label Class: 8+1


,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
0,0.158755,-0.460666,11.118750,0.380183,-0.231754,-0.231754,0.0,0.0,0.0,0.0,...,-0.588789,-0.797205,-1.461175,-1.502993,-0.863778,-0.588342,-0.231288,0.637615,-1.503092,0
1,1.524459,-0.457775,8.357143,1.501086,-0.231678,-0.231678,0.0,0.0,0.0,0.0,...,-0.490582,-0.785343,1.553021,1.532797,-0.790863,-0.489910,-0.228472,0.995545,1.540535,0
2,1.306708,-0.408941,8.200000,-0.631581,-0.231694,-0.231694,0.0,0.0,0.0,0.0,...,4.028637,4.377559,-1.461175,-1.502993,4.133899,4.026441,2.985967,0.746550,-1.503092,0


Sampler: ClusterCentroids / Label Class: 1+1


,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
0,0.158755,-0.460666,11.118750,0.380183,-0.231754,-0.231754,0.0,0.0,0.0,0.0,...,-0.588789,-0.797205,-1.461175,-1.502993,-0.863778,-0.588342,-0.231288,0.637615,-1.503092,0
1,1.524459,-0.457775,8.357143,1.501086,-0.231678,-0.231678,0.0,0.0,0.0,0.0,...,-0.490582,-0.785343,1.553021,1.532797,-0.790863,-0.489910,-0.228472,0.995545,1.540535,0
2,1.306708,-0.408941,8.200000,-0.631581,-0.231694,-0.231694,0.0,0.0,0.0,0.0,...,4.028637,4.377559,-1.461175,-1.502993,4.133899,4.026441,2.985967,0.746550,-1.503092,0


## X, y Train/Test Splitting

In [5]:
# Not storing test datasets in memory, will save as parquet and later load for testing.
X_train = {}
y_train = {}

print('Train/Test Shapes:')

for sampler in sampling_methods:
    X_train[sampler] = {}
    y_train[sampler] = {}
    
    for label_class in label_classes:
        X = datasets[sampler][label_class].drop('label', axis=1)
        y = datasets[sampler][label_class]['label']
        
        X_train_temp, X_test_temp, y_train_temp, y_test_temp = train_test_split(
            X, y, test_size=0.2, random_state=train_test_seed
        )
        
        print(f'\nSampler: {sampler} / Label Class: {label_class}')
        print(f'X_train: {X_train_temp.shape}, y_train: {y_train_temp.shape}, X_test: {X_test_temp.shape}, y_test: {y_test_temp.shape}')
        
        X_train[sampler][label_class] = X_train_temp
        y_train[sampler][label_class] = y_train_temp
        
        # Save test dataset
        pd.concat([X_test_temp, y_test_temp], axis=1).to_parquet(
            path=f'./test_datasets/{sampler}_{label_class}_test_dataset.parquet'
        )

# Whole datasets no longer used. Free memory.
del datasets

Train/Test Shapes:

Sampler: None / Label Class: 33+1
X_train: (1111526, 46), y_train: (1111526,), X_test: (277882, 46), y_test: (277882,)

Sampler: None / Label Class: 8+1
X_train: (1111526, 46), y_train: (1111526,), X_test: (277882, 46), y_test: (277882,)

Sampler: None / Label Class: 1+1
X_train: (1111526, 46), y_train: (1111526,), X_test: (277882, 46), y_test: (277882,)

Sampler: RandomOverSampler / Label Class: 33+1
X_train: (5839350, 46), y_train: (5839350,), X_test: (1459838, 46), y_test: (1459838,)

Sampler: RandomOverSampler / Label Class: 8+1
X_train: (6473779, 46), y_train: (6473779,), X_test: (1618445, 46), y_test: (1618445,)

Sampler: RandomOverSampler / Label Class: 1+1
X_train: (2170537, 46), y_train: (2170537,), X_test: (542635, 46), y_test: (542635,)

Sampler: RandomUnderSampler / Label Class: 33+1
X_train: (870, 46), y_train: (870,), X_test: (218, 46), y_test: (218,)

Sampler: RandomUnderSampler / Label Class: 8+1
X_train: (2579, 46), y_train: (2579,), X_test: (645, 4

# Evaluators

In [6]:
# Get all evaluators
evaluators = []
for evaluator_type in evaluator_types:
    for sampler in sampling_methods:
        for label_class in label_classes:
            evaluators.append((evaluator_type, sampler, label_class))

# Get list of pretrained evaluators
pretrained_evaluators = [filename for filename in os.listdir(evaluator_directory) if filename.endswith('.joblib')]

for i, evaluator in enumerate(pretrained_evaluators):
    pretrained_evaluators[i] = tuple(evaluator[:-7].split('_'))

# Get list of untrained evaluators
untrained_evaluators = [evaluator for evaluator in evaluators if evaluator not in pretrained_evaluators]

print(f'All evaluators: {len(evaluators)}')
print(f'Pretrained evaluators: {len(pretrained_evaluators)}')
print(f'Untrained evaluators: {len(untrained_evaluators)}')

All evaluators: 15
Pretrained evaluators: 105
Untrained evaluators: 0


## Training

In [7]:
# Only train evaluators not found in ./trained_evaluators
for i, (evaluator_type, sampler, label_class) in enumerate(untrained_evaluators):
    print(f'{i+1}/{len(untrained_evaluators)} : {evaluator_type}, {sampler}, {label_class}')
    match evaluator_type:
        case 'XGBoost':
            from xgboost import XGBClassifier
            evaluator_model = XGBClassifier()
            
        case 'LogisticRegression':
            from sklearn.linear_model import LogisticRegression
            evaluator_model = LogisticRegression(random_state=evaluator_seed, n_jobs=-1)
            
        case 'Perceptron':
            from sklearn.linear_model import Perceptron
            evaluator_model = Perceptron(random_state=evaluator_seed, n_jobs=-1)
            
        case 'AdaBoost':
            from sklearn.ensemble import AdaBoostClassifier
            evaluator_model = AdaBoostClassifier(random_state=evaluator_seed, algorithm='SAMME')
            
        case 'RandomForest':
            from sklearn.ensemble import RandomForestClassifier
            evaluator_model = RandomForestClassifier(random_state=evaluator_seed, n_jobs=-1)
            
        case 'DeepNeuralNetwork':
            from sklearn.neural_network import MLPClassifier
            evaluator_model = MLPClassifier(random_state=evaluator_seed)
            
        case 'KNearestNeighbor':
            from sklearn.neighbors import KNeighborsClassifier
            evaluator_model = KNeighborsClassifier(n_jobs=-1)
            
        case _:
            print(f'Invalid evaluator model: {evaluator_type}')
    
    
    if evaluator_type == 'XGBoost' and label_class == '1+1':
        evaluator_model = XGBClassifier(objective='binary:logistic')
            
    print(f'{time.time()} : Training {evaluator_type} on {sampler} balanced data with {label_class} label classes')
    evaluator_model.fit(X_train[sampler][label_class], y_train[sampler][label_class])
    print(f'{time.time()} : Training completed.\n')
    
    # Saving evaluator to file
    dump(evaluator_model, filename=f'./trained_evaluators/{evaluator_type}_{sampler}_{label_class}.joblib')

In [8]:
# Free training memory
try:
    del evaluator_model
except NameError:
    print('No evaluators to delete')
del X_train
del y_train

No evaluators to delete


## Testing

In [ ]:
# Load previous metrics dataframe
try: 
    df_metrics = pd.read_json(path_or_buf=metrics_directory+'/sampling_metrics.json', orient='index')
    
except FileNotFoundError:
    df_metrics = pd.DataFrame(columns=['Sampler', 'Label Classes', 'Evaluator', 'Test Duration', 'Accuracy', 'Precision', 'Recall', 'F1'])

# Run prediction
for (evaluator_type, sampler, label_class) in evaluators:
    # Load evaluator
    try:
        evaluator = load(filename=f'./trained_evaluators/{evaluator_type}_{sampler}_{label_class}.joblib')
    
    except FileNotFoundError:
        print(f'EVALUATOR NOT FOUND: ./trained_evaluators/{evaluator_type}_{sampler}_{label_class}.joblib\n')
    
    else:
        # Load testing dataset
        df_test = pd.read_parquet(path=f'./test_datasets/{sampler}_{label_class}_test_dataset.parquet')
        
        X_test = df_test.drop('label', axis=1)
        y_test = df_test['label']
            
        print(f'{time.time()} : Predicting {evaluator_type} on {sampler} sampled data with {label_class} classes')
        
        start_time = time.time()
        y_pred = evaluator.predict(X_test)
        test_duration = time.time() - start_time
        
        # Gather metrics  
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='macro', zero_division=0.0)
        recall = recall_score(y_test, y_pred, average='macro')
        f1 = f1_score(y_test, y_pred, average='macro')
        
        evaluator_metrics = [sampler, label_class, evaluator_type, test_duration, accuracy, precision, recall, f1]
                
        # Add metrics to dataframe and display
        update_row = df_metrics.loc[(df_metrics['Sampler'] == sampler) &
                                    (df_metrics['Label Classes'] == label_class) &
                                    (df_metrics['Evaluator'] == evaluator_type)]
        
        if update_row.empty:    
            # No previous record
            df_metrics.loc[len(df_metrics.index)] = evaluator_metrics
            
            print(f'{evaluator_type} / {sampler} / {label_class} Metrics')
            display(df_metrics.loc[len(df_metrics.index)-1])
        
        else:   
            # Previous record exists
            update_row = evaluator_metrics
        
            print(f'{evaluator_type} / {sampler} / {label_class} Metrics')
            display(df_metrics.loc[(df_metrics['Sampler'] == sampler) &
                                    (df_metrics['Label Classes'] == label_class) &
                                    (df_metrics['Evaluator'] == evaluator_type)])

1715522431.3831902 : Predicting KNearestNeighbor on 33+1 classes
New Record
KNearestNeighbor / None / 33+1 Metrics


Sampler                      None
Label Classes                33+1
Evaluator        KNearestNeighbor
Test Duration          178.839612
Accuracy                 0.946546
Precision                0.656483
Recall                   0.623892
F1                       0.631163
Name: 90, dtype: object

1715522610.8434405 : Predicting KNearestNeighbor on 8+1 classes
New Record
KNearestNeighbor / None / 8+1 Metrics


Sampler                      None
Label Classes                 8+1
Evaluator        KNearestNeighbor
Test Duration          177.430614
Accuracy                 0.954509
Precision                0.694458
Recall                   0.622671
F1                       0.638203
Name: 91, dtype: object

1715522788.7708628 : Predicting KNearestNeighbor on 1+1 classes
New Record
KNearestNeighbor / None / 1+1 Metrics


Sampler                      None
Label Classes                 1+1
Evaluator        KNearestNeighbor
Test Duration          177.506453
Accuracy                 0.991896
Precision                0.897603
Recall                   0.940536
F1                       0.917904
Name: 92, dtype: object

1715522967.8579066 : Predicting KNearestNeighbor on 33+1 classes


## Save Metrics to File

In [ ]:
df_metrics['Sampler'] = pd.Categorical(df_metrics['Sampler'], categories=sampler_categories)
df_metrics['Label Classes'] = pd.Categorical(df_metrics['Label Classes'], categories=label_categories)
df_metrics['Evaluator'] = pd.Categorical(df_metrics['Evaluator'], categories=evaluator_categories)

df_metrics.sort_values(['Sampler', 'Label Classes', 'Evaluator'], inplace=True)

df_metrics.to_json(path_or_buf=metrics_directory+'/sampling_metrics.json', orient='index')

display(df_metrics)